## Deep Research

One of the classic cross-business Agentic use cases! This is huge.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">A Deep Research agent is broadly applicable to any business area, and to your own day-to-day activities. You can make use of this yourself!
            </span>
        </td>
    </tr>
</table>

I hope you find this lab useful! I would love to connect on LinkedIn: https://www.linkedin.com/in/kyranank/

### Upgrades to Original System (Rough Notes)

- When given a query, come up with 3 questions that will help answer the query
    - Then come up with a search query to answer each question
- Incorporate handoffs and using agents as tools
    - Agents as Tools: 1 Google Search agent and 1 OpenAI Web Search agent. The research manager will decide if more queries are needed (and can call planner_agent again), or if need to run again.
    - Agents as Tools: 1 question formulator agent and 1 search planner agent which get used by the research manager in sequence.
    - Handoff the research to the writer agent
    - Handoff the report to the editor agent

PLAN
- Create & test question_formulator_agent
    - Given a query, formulate X research questions to help answer the query
    - Test with a query
    - Convert to .as_tool()
- Adjust previous planner_agent & test
    - Use the question_formulator_agent as a tool to generate questions
    - Create 1 search query to answer each 1 of the questions
    - Test with simulated questions
    - Convert to .as_tool()
- Create & test gemini_search_agent
    - Someone has a version in the community contributions folder
    - Take the list of queries and perform a search for each one
    - Test with simulated query list
    - Convert to .as_tool()
- Adjust previous openai_search_agent & test
    - Take the list of queries and perform a search for each one
    - Test with simulated query list
    - Convert to .as_tool()
- Create an editor_agent
    - Receives the report and edits it to a professional level
    - Returns the same as writer_agent
- Adjust the writer_agent
    - Receives the research, writes a report, also returns a markdown report
    - Handsoff to editor_agent
- Create a research_manager_agent
    - Given a query use the planner_agent to generate search queries to research the query
    - Next use the gemini_search_agent and openai_search_agents to conduct the search queries. They should each be used once for every search query. That way you receive two different results to consider for each query for the final research report. If you believe more information is needed to address the query, you can use the planner_agent only one additional time followed by the gemini_search_agent and the gemini_search_agent to gather additional research. You will then handoff this research to the writer_agent to write a report.

In [2]:
from agents import Agent, WebSearchTool, trace, Runner, function_tool, handoff, set_default_openai_client, set_tracing_export_api_key
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown
from agents import OpenAIChatCompletionsModel
from openai import AsyncOpenAI, AsyncAzureOpenAI
import requests
from bs4 import BeautifulSoup

In [12]:
load_dotenv(override=True)

openai_client = AsyncAzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-08-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment="gtp-4o"
)

openai = os.getenv("OPENAI_API_KEY")

set_default_openai_client(openai_client);
set_tracing_export_api_key(openai);

google_api_key = os.getenv('GOOGLE_API_KEY')

gemini = AsyncOpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.5-flash"

### Question Formulator Agent
- Create & test question_formulator_agent
    - Given a query, formulate X research questions to help answer the query
    - Test with a query
    - Convert to .as_tool()

In [4]:
INSTRUCTIONS = "You are a helpful research assistant. Given a query, formulate a set of research questions \
that when answered will help answer or address the query. Output 3 questions to research."

class ResearchQuestionItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this question is important to address the query.")
    question: str = Field(description="The research question to use to conduct further research regarding the query.")

class ResearchQuestionPlan(BaseModel):
    questions: list[ResearchQuestionItem] = Field(description="A list of research questions to research to best answer the query.")

question_formulator_agent = Agent(
    name="Question Formulator Agent",
    instructions=INSTRUCTIONS,
    model=OpenAIChatCompletionsModel(openai_client=openai_client, model="gpt-4o"),
    output_type=ResearchQuestionPlan,
)

In [8]:
message = "Latest AI Agent frameworks in 2025"

with trace("Test - Question Formulator Agent"):
    result = await Runner.run(question_formulator_agent, message)

print(result.final_output)

questions=[ResearchQuestionItem(reason='Understanding the most current frameworks will provide insights into the latest capabilities, trends, and tools available for developing AI agents in 2025.', question='What are the most popular AI agent frameworks available in 2025?'), ResearchQuestionItem(reason='Comparing the features of these frameworks will help in identifying which framework might be best suited for specific use cases or technical requirements.', question='What are the key features and differences among the leading AI agent frameworks in 2025?'), ResearchQuestionItem(reason='Exploring adoption trends among developers and companies can reveal the frameworks that are gaining traction in the industry, which may reflect their effectiveness or ease of use.', question='Which AI agent frameworks are gaining the most adoption and why in 2025?')]


In [6]:
description = "Generate research questions"

tool_question_formulator = question_formulator_agent.as_tool(tool_name="question_formulator_agent", tool_description=description)

tools_for_planner_agent = [tool_question_formulator]

tools_for_planner_agent

[FunctionTool(name='question_formulator_agent', description='Generate research questions', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'question_formulator_agent_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000246C94367A0>, strict_json_schema=True, is_enabled=True)]

### Planner Agent
- Adjust previous planner_agent & test
    - Use the question_formulator_agent as a tool to generate questions
    - Create 1 search query to answer each 1 of the questions
    - Test with simulated questions
    - Convert to .as_tool()

In [9]:
INSTRUCTIONS = "You are a helpful research assistant. Given a query you will use the question_formulator_agent as a tool to \
come up with a set of questions. Your role is to create 1 web search for each question to perform to best answer the query. \
Output the web search terms to query for, there should be exactly as many web search queries as questions you received."

class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query and will answer the question.")
    query: str = Field(description="The search term to use for the web search.")

class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")

planner_agent = Agent(
    name="Planner Agent",
    instructions=INSTRUCTIONS,
    model=OpenAIChatCompletionsModel(openai_client=openai_client, model="gpt-4o"),
    output_type=WebSearchPlan,
    tools=tools_for_planner_agent
)

In [10]:
message = "Latest AI Agent frameworks in 2025"

with trace("Test - Planner Agent"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='Identifying the latest AI agent frameworks will provide a comprehensive understanding of the most advanced tools being utilized in 2025.', query='latest AI agent frameworks 2025'), WebSearchItem(reason='Comparing with popular frameworks used previously helps to understand the evolution, improvements, and possible shifts in preferences.', query='AI agent frameworks 2025 vs 2023'), WebSearchItem(reason='Understanding these features can help determine the strengths and suitability of these frameworks for specific applications.', query='features of AI agent frameworks 2025')]


In [11]:
description = "Generate web search terms"

tool_planner_agent = planner_agent.as_tool(tool_name="planner_agent", tool_description=description)
tools_for_research_manager_agent = [tool_planner_agent]
tools_for_research_manager_agent

[FunctionTool(name='planner_agent', description='Generate web search terms', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'planner_agent_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000246CD2B5DA0>, strict_json_schema=True, is_enabled=True)]

### Gemini Search Agent
- Create & test gemini_search_agent
    - Someone has a version in the community contributions folder
    - Take the list of queries and perform a search for each one
    - Test with simulated query list
    - Convert to .as_tool()

In [13]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai"
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CUSTOM_SEARCH_API_KEY = os.getenv("GOOGLE_CUSTOM_SEARCH_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")

gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=os.getenv('GOOGLE_API_KEY'))
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash",openai_client=gemini_client)

In [14]:
# Tool for Web Search - Google doesn't offer a remote WebSearchTool like OpenAI
# Need to set up Google Programmable Search / Custom Search API and get a key + search engine ID

@function_tool
async def google_web_search(query: str) -> str:
    """
    Performs a Google Search and returns a string of concatenated result snippets.
    """
    endpoint = "https://www.googleapis.com/customsearch/v1"
    params = {
        "key": GOOGLE_CUSTOM_SEARCH_API_KEY,
        "cx": GOOGLE_CSE_ID,
        "q": query,
        "num": 5,  # Number of results to fetch
    }

    response = requests.get(endpoint, params=params)
    if response.status_code != 200:
        raise RuntimeError(f"Google Search API error {response.status_code}: {response.text}")

    data = response.json()
    items = data.get("items", [])

    if not items:
        return "No results found."

    # Combine title and snippet for each result
    result_texts = []
    for item in items:
        title = item.get("title", "").strip()
        snippet = item.get("snippet", "").strip()
        link = item.get("link", "").strip()
        result_texts.append(f"{title}\n{snippet}\n{link}")

    summary = "\n\n".join(result_texts)
    return summary

In [15]:
INSTRUCTIONS = "You are a research assistant. Given a web search term, you search the web using the google_web_search tool \
for that term and produce a concise summary of the results. The summary must 5-8 paragraphs and between 500-1000 words \
words. Capture the main points. Write succintly, add evidence/facts, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

gemini_search_agent = Agent(
    name="Gemini Search Agent",
    instructions=INSTRUCTIONS,
    tools=[google_web_search],
    model=gemini_model,
    model_settings=ModelSettings(tool_choice="required"),
)

In [16]:
message = "notable AI agent frameworks introduced or updated in 2025"

with trace("Test - Gemini Search Agent"):
    result = await Runner.run(gemini_search_agent, message)

display(Markdown(result.final_output))

The search results provide an overview of AI agent frameworks that were prominent or updated in 2025. Key frameworks mentioned include LangChain, Transformers Agents, and others. The results also highlight the expanding nature of AI agent frameworks and their evolving capabilities.

**Key AI Agent Frameworks in 2025:**

*   **LangChain:**  Ecosystem constantly growing with new community-contributed elements, tools, and connectors being introduced regularly.

*   **Transformers Agents:** Introduced by Hugging Face, leveraging the power of transformer models.

**General Trends and Concepts:**

*   **Agentic Architecture:** Refers to the structure and design of agentic AI frameworks, shaping the workflow structure to automate AI models.
*   **Growing Ecosystem:** The field of AI agent frameworks is continuously expanding and evolving, with new frameworks and updates emerging regularly.

The search results suggest a dynamic landscape for AI agent frameworks in 2025, characterized by ongoing development, new introductions, and increasing adoption in various applications.


In [17]:
description = "Search the web using Google Search"

tool_google_search = gemini_search_agent.as_tool(tool_name="gemini_search_agent", tool_description=description)

In [18]:
tools_for_research_manager_agent.append(tool_google_search)
tools_for_research_manager_agent

[FunctionTool(name='planner_agent', description='Generate web search terms', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'planner_agent_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000246CD2B5DA0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='gemini_search_agent', description='Search the web using Google Search', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'gemini_search_agent_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000246CD2B7E20>, strict_json_schema=True, is_enabled=True)]

### OpenAI Search Agent
- Adjust previous openai_search_agent & test
    - Take the list of queries and perform a search for each one
    - Test with simulated query list
    - Convert to .as_tool()
    

In [23]:
INSTRUCTIONS = "You are a research assistant. Given a web search term, you search the web \
for that term and produce a concise summary of the results. The summary must 5-8 paragraphs and between 500-1000 words \
words. Capture the main points. Write succintly, add evidence/facts, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

openai_search_agent = Agent(
    name="Search Agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [24]:
message = "notable AI agent frameworks introduced or updated in 2025"

with trace("Search"):
    result = await Runner.run(openai_search_agent, message)

display(Markdown(result.final_output))

Error getting response: Unsupported data type. (request_id: None)


BadRequestError: Unsupported data type

In [25]:
description = "Search the web using OpenAI's WebSearchTool"

tool_openai_search = openai_search_agent.as_tool(tool_name="openai_search_agent", tool_description=description)

In [26]:
tools_for_research_manager_agent.append(tool_openai_search)
tools_for_research_manager_agent

[FunctionTool(name='planner_agent', description='Generate web search terms', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'planner_agent_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000246CD2B5DA0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='gemini_search_agent', description='Search the web using Google Search', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'gemini_search_agent_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000246CD2B7E20>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='openai_search_agent', description="Search the web using OpenAI's WebSearchTool", params_json_schema={'prope

### Editor Agent
- Create an editor_agent
    - Receives the report and edits it to a professional level
    - Returns the same as writer_agent

In [27]:
class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")
    markdown_report: str = Field(description="The final report")
    follow_up_questions: list[str] = Field(description="Suggested topics to research further")

In [30]:
INSTRUCTIONS = (
    "You are a senior editor tasked with editing and revising a cohesive report for a research query. "
    "You will be provided with the report data including a short summary, full markdown_report, and some follow_up_questions\n"
    "You should first review the report, analyze the structure, correct grammar, spelling, sentence structure, etc. for profressionalism"
    "Then, create a revised report using the original report as a basis and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed with FULL SENTENCES."
    "This is a research report, aim for 10-15 pages of content, AT LEAST 2000 words, maximum of 5000 words."
)

editor_agent = Agent(
    name="Editor Agent",
    instructions=INSTRUCTIONS,
    model=OpenAIChatCompletionsModel(openai_client=openai_client, model="gpt-4o"),
    output_type=ReportData
)

### Writer Agent
- Adjust the writer_agent
    - Receives the research, writes a report, also returns a markdown report
    - Handsoff to editor_agent

In [31]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report using the research as a basis and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 10-15 pages of content, at least 2000 words."
    "Once you have gathered all research results, hand off the complete set of research materials to the `editor_agent`.\n"
    "The `writer_agent` will be responsible for synthesizing the information into a final research report."
)

writer_agent = Agent(
    name="Writer Agent",
    instructions=INSTRUCTIONS,
    model=OpenAIChatCompletionsModel(openai_client=openai_client, model="gpt-4o"),
    output_type=ReportData,
    handoffs=[handoff(editor_agent)]
)

### Research Manager Agent
- Create a research_manager_agent
    - Given a query use the planner_agent to generate search queries to research the query
    - Next use the gemini_search_agent and openai_search_agents to conduct the search queries. They should each be used once for every search query. That way you receive two different results to consider for each query for the final research report. If you believe more information is needed to address the query, you can use the planner_agent only one additional time followed by the gemini_search_agent and the gemini_search_agent to gather additional research. You will then handoff this research to the writer_agent to write a report.

In [32]:
INSTRUCTIONS = """
You are a Research Manager Agent responsible for coordinating a multi-step research process to produce high-quality research reports.

Follow this workflow precisely:

1. **Planning Phase**
   - When given a research query, always start by using the `planner_agent`.
   - Use the `planner_agent` to generate a set of structured search queries. The planner will return these queries as a Pydantic model.

2. **Initial Research Phase**
   - For each search query provided by the planner, conduct research using both of the following agents:
     - `gemini_search_agent`
     - `openai_search_agent`
   - You must use each of these search agents exactly once per search query. This ensures you gather two different perspectives for every query.

3. **Supplemental Research (if necessary)**
   - After reviewing the initial results, if you determine that additional information is required to adequately address the original research query, you may proceed to collect more research.
   - To do this, you may call the `planner_agent` **only one additional time** to generate more search queries.
   - For each additional search query, again use both:
     - `gemini_search_agent`
     - `openai_search_agent`
   - Do not exceed this single additional round of planning and research.

4. **Report Handoff**
   - Once you have gathered all research results, hand off the complete set of research materials to the `writer_agent`.
   - The `writer_agent` will be responsible for synthesizing the information into a final research report.

Important Guidelines:
- Always perform the research workflow in the sequence described: planner → search agents → (optional, only allowed to call 1 additional time) planner → search agents → writer.
- Do not skip any steps or call the search agents before the planner.
- DO NOT call the planner more than two (2) times TOTAL per query. You should only call it a maximum of 1 initial time and 1 additional time, for a total of 2 times.
- Ensure all gathered results are organized clearly before passing them to the writer agent.
"""

In [33]:
research_manager_agent = Agent(
    name="Research Manager Agent",
    instructions=INSTRUCTIONS,
    tools=tools_for_research_manager_agent,
    model=OpenAIChatCompletionsModel(openai_client=openai_client, model="gpt-4o"),
    model_settings=ModelSettings(tool_choice="required"),
    handoffs=[handoff(writer_agent)]
)

In [34]:
# RUN AGENTIC SYSTEM HERE!

message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    final_result = await Runner.run(research_manager_agent, message)
    print("Report Complete!")

Error getting response: Unsupported data type. (request_id: None)
Error getting response: Unsupported data type. (request_id: None)
Error getting response: Unsupported data type. (request_id: None)
Error getting response: Unsupported data type. (request_id: None)
Error getting response: Unsupported data type. (request_id: None)
Error getting response: Unsupported data type. (request_id: None)


Report Complete!


In [35]:
display(Markdown(final_result.final_output.short_summary))

The AI agent frameworks of 2025 are characterized by their focus on collaborative teamwork, integration with existing systems, flexibility, and adaptability. Frameworks such as Microsoft AutoGen, LangChain, and CrewAI lead the development landscape. Developer preferences are influenced by factors such as ease of use, level of customization, and specific project requirements.

In [36]:
display(Markdown(final_result.final_output.markdown_report))

# The Latest AI Agent Frameworks in 2025

## Introduction
In 2025, AI agent frameworks have evolved considerably to meet the growing demands of sophisticated AI systems. This report delves into the most commonly used frameworks, their key features, and why developers favor certain platforms. This comprehensive analysis serves as a guide for those looking to understand and possibly implement the latest advancements in AI agent technology.

## Common AI Agent Frameworks in 2025

### Key Frameworks
The AI technology landscape has seen several significant frameworks gaining popularity by 2025, each offering unique features and capabilities:

1. **Microsoft AutoGen**: This open-source framework is particularly suited for creating and orchestrating AI agents and multi-agent systems. It benefits from seamless integration with Microsoft tools, enhancing its attractiveness for enterprises already invested in Microsoft's ecosystem.

2. **LangChain**: Renowned for building effective AI workflows, LangChain emphasizes flexibility and ease of use, enabling developers to construct complex, automated processes efficiently.

3. **LangGraph**: Complementary to LangChain, this framework specializes in advanced state management, which is critical for AI agents requiring robust, dynamic interaction management.

4. **Phidata**: Known as an adaptive agent framework, Phidata offers customization and adaptability, making it a preferred choice for projects demanding precision and flexibility.

5. **CrewAI**: This framework focuses on teamwork, facilitating the creation of AI agents that can effectively collaborate within teams, highlighting a shift towards more dynamic, group-oriented AI development.

6. **Microsoft Semantic Kernel**: Another product from Microsoft, emphasizing both semantic capabilities and automated AI processes, showing the tech giant's commitment to comprehensive AI solutions.

7. **Rasa**: Continuing its focus on conversational AI, Rasa remains a pivotal tool for developers creating interactive AI agents that require natural language processing and dialogue management.

8. **JADE (Java Agent Development Framework)**: A stalwart in the field, JADE provides a solid Java-based environment for developing a wide range of AI agents.

## Key Features and Innovations

### Fundamental Attributes
Current frameworks embody a set of core features and innovations designed to enhance AI development:

- **Integration Capabilities**: Modern frameworks focus heavily on integration with existing systems to facilitate seamless deployment and usage across various platforms.

- **Event-driven Architectures**: Many frameworks are adopting event-driven paradigms to improve responsiveness and efficiency, crucial for real-time AI operations.

- **Pythonic Design**: The use of Python annotations and constructs leads to more intuitive and structured development processes, supporting a wide base of developers familiar with Python.

- **Flexibility and Adaptability**: As the AI landscape is ever-evolving, frameworks that offer flexibility and adaptability are highly valued. They ensure that developments remain relevant and capable of integrating future technologies.

### Advanced Features
Some frameworks, such as OpenAI Gym, specialize in reinforcement learning to train AI agents more effectively, providing tools that are particularly useful for developing high-performance AI systems capable of complex decision-making.

## Developer Preferences and Trends

Developers in 2025 base their framework preferences on various qualitative and technical factors:

- **Ease of Use**: Frameworks that offer simplified processes or intuitive interfaces enable developers to focus on creativity and problem-solving rather than technical hurdles.

- **Level of Control**: Advanced frameworks like LangGraph provide deeper, more refined control over agent behaviors and interactions, appealing to seasoned developers dealing with intricate AI systems.

- **Project Requirements**: The choice of framework often aligns with project-specific needs, such as the necessity for multi-agent collaboration or particular integration abilities.

- **Community and Support**: Strong community backing and comprehensive support systems are influential in choosing a particular framework, ensuring that developers have access to troubleshooting and advice.

## Conclusion
The AI agent frameworks of 2025 reflect a blend of innovation, practicality, and collaborative capabilities. As these frameworks continue to evolve, they advance the possibilities in AI development, offering developers a range of tools that cater to both the broad and the specific needs of modern applications. Understanding these frameworks and their capabilities is essential for leveraging the full potential of AI in 2025.

## References
Gathering all references from current research and forums, predicted technological advancements, and statements from key technology developers.

### Trace

https://platform.openai.com/traces

In [4]:
from agents.extensions.visualization import draw_graph

draw_graph(research_manager_agent)

ModuleNotFoundError: No module named 'graphviz'

In [38]:
from agents.extensions.visualization import draw_graph
draw_graph(writer_agent)

ModuleNotFoundError: No module named 'graphviz'

In [39]:
from agents.extensions.visualization import draw_graph
draw_graph(editor_agent)

ModuleNotFoundError: No module named 'graphviz'